# Initializing

## Libraries

In [1]:
# Utilities
import pandas as pd
import time
import numpy as np

# Cross-Validation Techniques
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut

# Model used
from sklearn.tree import DecisionTreeClassifier

# Metric used (Accuracy)
from sklearn.metrics import accuracy_score

# Dataset used
from sklearn.datasets import load_breast_cancer

## Loading the data

In [2]:
# Loading the data
data_wdbc = load_breast_cancer()

# Creating the dataframe with the features
X = pd.DataFrame(data = data_wdbc.data, columns = data_wdbc.feature_names)

# Defining the target
y = data_wdbc.target

print('Shape of the data:', X.shape)

Shape of the data: (569, 30)


In [3]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# Holdout Cross-Validation

In [4]:
# Spliting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

print('Shape of the Training set:', X_train.shape)
print('Shape of the Test set:', X_test.shape)

Shape of the Training set: (398, 30)
Shape of the Test set: (171, 30)


# K-Fold Cross-Validation

In [5]:
# Creating the K-Fold ojbect
kf = KFold(n_splits = 3)

# Performing the K-Fold Cross-Validation
# Each split represents one subset
for train_index, test_index in kf.split(X):
    
    # Obtaining the index for the training and test sets for the subset
    print("First five train index:", train_index[0:5], "First five test index:", test_index[0:5])
    print('Shape of the Training set:', train_index.shape, 'Shape of the Test set:', test_index.shape, '\n')

First five train index: [190 191 192 193 194] First five test index: [0 1 2 3 4]
Shape of the Training set: (379,) Shape of the Test set: (190,) 

First five train index: [0 1 2 3 4] First five test index: [190 191 192 193 194]
Shape of the Training set: (379,) Shape of the Test set: (190,) 

First five train index: [0 1 2 3 4] First five test index: [380 381 382 383 384]
Shape of the Training set: (380,) Shape of the Test set: (189,) 



# Leave-One-Out Cross-Validation

In [6]:
# Creating the Leave-One-Out ojbect
loo = LeaveOneOut()

# Auxiliar variable to stop the prints
break_aux = 0

# Performing the Leave-One-Out Cross-Validation
for train_index, test_index in loo.split(X):
    
    # Stop printing after the third iteraction
    if break_aux < 3:
        print("First five train index:", train_index[0:5], "Test index:", test_index)
        break_aux = break_aux + 1

First five train index: [1 2 3 4 5] Test index: [0]
First five train index: [0 2 3 4 5] Test index: [1]
First five train index: [0 1 3 4 5] Test index: [2]


# Classification Performance
## Holdout CV

In [7]:
# Initializing the Decision Tree object with default parameters
tree = DecisionTreeClassifier(random_state = 1234)

# Calculating the run time
holdout_start_time = time.time()

# Spliting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)

# Training the classifier
tree.fit(X_train, y_train)

# Making the predictions
y_pred = tree.predict(X_test)

# Calculating the accuracy
acc_holdout = accuracy_score(y_test, y_pred)

holdout_performance_time = time.time() - holdout_start_time
print('Accuracy obtained:', acc_holdout*100)
print('Run Time:', holdout_performance_time)

Accuracy obtained: 92.98245614035088
Run Time: 0.009000539779663086


## 3-Fold CV

In [8]:
# Initializing the Decision Tree object with default parameters
tree = DecisionTreeClassifier(random_state = 1234)

# List to store the results
acc_list = []

# Calculating the run time
kfold_start_time = time.time()

# Creating the K-Fold ojbect
kf = KFold(n_splits = 3)

# Performing the K-Fold Cross-Validation
# Each split represents one subset
for train_index, test_index in kf.split(X):
    
    # Obtaining the training and test subsets
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Training the model
    tree.fit(X_train, y_train)
    
    # Making the predictions
    y_pred = tree.predict(X_test)

    # Calculating the accuracy
    acc = accuracy_score(y_test, y_pred)
    
    # Saving the result in the list
    acc_list.append(acc)
    
# Calculating the mean of the results
acc_kfold = np.mean(acc_list)

kfold_performance_time = time.time() - kfold_start_time
print('Accuracy obtained:', acc_kfold*100)
print('Run Time:', kfold_performance_time)

Accuracy obtained: 90.15408892601874
Run Time: 0.014000654220581055


## Leave-One-Out CV

In [9]:
# Initializing the Decision Tree object with default parameters
tree = DecisionTreeClassifier(random_state = 1234)

# List to store the results
acc_list = []

# Calculating the run time
loo_start_time = time.time()

# Performing the Leave-One-Out Cross-Validation
for train_index, test_index in loo.split(X):
    
    # Obtaining the training and test subsets
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Training the model
    tree.fit(X_train, y_train)
    
    # Making the predictions
    y_pred = tree.predict(X_test)

    # Calculating the accuracy
    acc = accuracy_score(y_test, y_pred)
    
    # Saving the result in the list
    acc_list.append(acc)
    
# Calculating the mean of the results
acc_loo = np.mean(acc_list)

loo_performance_time = time.time() - loo_start_time
print('Accuracy obtained:', acc_loo*100)
print('Run Time:', loo_performance_time)

Accuracy obtained: 91.56414762741653
Run Time: 3.4548919200897217


## Results

In [10]:
res = {'Accuracy': [acc_holdout*100, acc_kfold*100, acc_loo*100], 'Run Time [ms]': [holdout_performance_time, kfold_performance_time, loo_performance_time]}

df_res = pd.DataFrame(data = res, index = ['Holdout CV', '3-Fold CV', 'Leave-One-Out CV'])
df_res

,Accuracy,Run Time [ms]
Holdout CV,92.982456,0.009001
3-Fold CV,90.154089,0.014001
Leave-One-Out CV,91.564148,3.454892
